# Imports

In [13]:
# Object-oriented
from abc import ABC, abstractmethod, abstractproperty

# Math 
import random
import numpy as np
from scipy.spatial import distance

# System
import sys
import io

# Plotting, widgets
import seaborn as sns
from ipywidgets import *
import matplotlib.patches
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from IPython.display import HTML as html
sns.set(style="ticks")
plt.rcParams["font.family"] = "Arial"

# Set random seed
RANDOM_SEED = 0

In [10]:
# Importing from other ipynbs that encapsulate classes with low-level design
import ipynb
import importlib

import ipynb.fs.full.Firefly as Firefly
import ipynb.fs.full.World as World

# Each time the low-level code is changed, just run this to reload
importlib.reload(Firefly)
importlib.reload(World)

# Collection of all widgets for GUI

In [11]:
class WidgetCollection:
    def __init__(self, firefly_world):
        self.firefly_world = firefly_world
        
        # Layout style for many widgets
        self.layout = Layout(border='0px solid red', width='350px', height='30px')
        
        # Generates all widgets
        self.generate_collection()
        
    def generate_textbox(self):
        self.ticks = Text(description="Ticks:", layout=Layout(width='200px', height='30px', margin='0px 0px 0px 60px'))
        
    def generate_buttons(self):
        self.setup_button = Button(description="setup/reset")
        self.setup_button.style.button_color = 'lightgreen'
        
        self.go_button = Button(description="go")
        self.go_button.style.button_color = 'lightgreen'
        
    def generate_plots(self):
        self.plot_data = HTML()
        self.plot_simulation = HTML()

    def generate_sliders(self):
        self.num_males_slider = IntSlider(description="Number of fireflies:", value=30, min=1, max=100, layout=self.layout)
        self.num_males_slider.style.handle_color = 'lightgreen'
        
        self.step_size_slider = IntSlider(description="Step size:", value=1, min=1, max=5, layout=self.layout)
        self.step_size_slider.style.handle_color = 'lightgreen'

        self.num_steps_slider = IntSlider(description="Number of ticks:", value=50, min=10, max=300, layout=self.layout)
        self.num_steps_slider.style.handle_color = 'lightgreen'
        
        self.turning_angle_slider = FloatSlider(description="Turning angle width:", value=np.pi/8, min=np.pi/15, max=np.pi/5, layout=self.layout)
        self.turning_angle_slider.style.handle_color = 'lightgreen'
        
        self.flash_min_slider = IntSlider(description="Flashing interval min:", value=5, min=1, max=15, layout=self.layout)
        self.flash_min_slider.style.handle_color = 'lightgreen'
        
        self.flash_max_slider = IntSlider(description="Flashing interval max:", value=30, min=10, max=100, layout=self.layout)
        self.flash_max_slider.style.handle_color = 'lightgreen'
        
    def generate_checkbox(self):
        self.trackon_checkbox = Checkbox(description="Show track", layout=Layout(width='500px', margin='0px 0 -35px 0px'))
    
    def update_widgets(self):
        self.ticks.value = str(self.firefly_world.ticks)
        self.plot_data.value = str(self.firefly_world.plot_data())
        self.plot_simulation.value = str(self.firefly_world.plot_simulation(int(self.ticks.value)))
        self.num_males_slider.value = self.firefly_world.num_males
        self.step_size_slider.value = self.firefly_world.firefly_collection.step_size
        self.num_steps_slider.value = self.firefly_world.num_total_steps
        self.turning_angle_slider.value = self.firefly_world.firefly_collection.turning_angle_distribution
        self.flash_min_slider.value = self.firefly_world.firefly_collection.flash_interval_min
        self.flash_max_slider.value = self.firefly_world.firefly_collection.flash_interval_max
        self.trackon_checkbox.value = self.firefly_world.track_on
        
    def generate_collection(self):
        
        ''' All the above widgets into one with arrangement in boxes. '''
        
        self.generate_textbox()
        self.generate_buttons()
        self.generate_plots()
        self.generate_sliders()
        self.generate_checkbox()
        
        row1 = HBox([self.setup_button, self.go_button, self.trackon_checkbox])
        row2 = HBox([self.num_males_slider, self.step_size_slider])
        row3 = HBox([self.num_steps_slider, self.turning_angle_slider])
        row4 = HBox([self.flash_min_slider, self.flash_max_slider])
        row5 = HBox([self.plot_data], layout=Layout(height="350px", width="500px"))
        self.widgets = HBox([VBox([row1, row2, row3, row4, row5], layout=Layout(width="600px")), 
                        VBox([self.ticks, self.plot_simulation], layout=Layout(width="500px"))])
       
        self.update_widgets()
        
    def __call__(self):
        return self.widgets

# GUI to select parameter, run simulation, and see plots in real time

In [12]:
class SimulationGUI:
    
    def __init__(self, widgets_collection, firefly_world):
        self.widgets_collection = widgets_collection
        self.firefly_world = firefly_world
        
        self.gui = self.run_simulation()
    
    def update(self, args):
        if args["name"] != "value":
            return
        owner = args["owner"]
        name = args["name"]
        value = args["new"]

        if owner == self.widgets_collection.num_males_slider:
            self.firefly_world.num_males = value
        elif owner == self.widgets_collection.step_size_slider:
            self.firefly_world.step_size = value
        elif owner == self.widgets_collection.num_steps_slider:
            self.firefly_world.num_total_steps = value
        elif owner == self.widgets_collection.turning_angle_slider:
            self.firefly_world.turning_angle_distribution = value
        elif owner == self.widgets_collection.flash_min_slider:
            self.firefly_world.flash_interval_min = value
        elif owner == self.widgets_collection.flash_max_slider:
            self.firefly_world.flash_interval_max = value
        elif owner == self.widgets_collection.trackon_checkbox:
            self.firefly_world.track_on = value
        
    def setup(self, widgets):
        self.firefly_world.setup()
        self.widgets_collection.plot_data.value = self.firefly_world.plot_data()
        self.widgets_collection.plot_simulation.value = self.firefly_world.plot_simulation(int(self.widgets_collection.ticks.value))
        self.widgets_collection.ticks.value = str(self.firefly_world.ticks)
        
    def run(self, widgets):
        if self.firefly_world.state == "stopped":
            w = {"plot_data": self.widgets_collection.plot_data, 
                 "ticks": self.widgets_collection.ticks, 
                 "plot_simulation": self.widgets_collection.plot_simulation} 
            self.firefly_world.run(w)
            
    def run_simulation(self):
        self.widgets_collection.num_males_slider.observe(self.update)         
        self.widgets_collection.step_size_slider.observe(self.update)
        self.widgets_collection.num_steps_slider.observe(self.update)
        self.widgets_collection.turning_angle_slider.observe(self.update)
        self.widgets_collection.flash_min_slider.observe(self.update)
        self.widgets_collection.flash_max_slider.observe(self.update)
        self.widgets_collection.trackon_checkbox.observe(self.update)

        self.widgets_collection.setup_button.on_click(self.setup)
        self.widgets_collection.go_button.on_click(self.run)
    
        return self.widgets_collection